In [ ]:
import pandas as pd

In [ ]:
my_dict = {
  "status": "success",
  "data": {
    "resultType": "matrix",
    "result": [
      {
        "metric": {
          "data0": "T",
          "data1": "O"
        },
        "values": [
          [
            90,
            "0"
          ],
          [
            80,
            "0"
          ]
        ]
      },
      {
        "metric": {
          "data0": "K",
          "data1": "S"
        },
        "values": [
          [
            70,
            "0"
          ],
          [
            60,
            "0"
          ]
        ]
      },
      {
        "metric": {
          "data2": "J",
          "data3": "O"
        },
        "values": [
          [
            50,
            "0"
          ],
          [
            40,
            "0"
          ]
        ]
      }
    ]
  }
}

In [ ]:
my_dict.keys()

In [ ]:
df1 = pd.DataFrame.from_records(my_dict['data']['result']).join(pd.DataFrame(df['metric'].to_list())).drop('metric', axis=1)

In [ ]:
%%time
# pd.json_normalize way
df1 = pd.json_normalize(my_dict['data']['result'])
df1.columns = df1.columns.str.strip('metric.')

In [ ]:
%%time
# another way, but slower with this example
df1 = pd.DataFrame.from_records(my_dict['data']['result']).join(pd.DataFrame(df['metric'].to_list())).drop('metric', axis=1)

In [ ]:
df2 = df1.explode('values')
df2[['time', 'value']] = df2['values'].to_list()
df2 = df2.drop('values', axis=1)

# Dask way

In [ ]:
import json
import dask.bag as db
import dask.dataframe as dd

In [ ]:
with open('/tmp/data.json', 'w') as f:
    json.dump(my_dict, f)

In [ ]:
my_bag = db.read_text(
    '/tmp/data.json' # or can use data_*.json for multiple files
).map(
    json.loads # lazy loading
).pluck('data').pluck('result').flatten() # only need the values from ['data']['result'] keys

In [ ]:
# if you don't provide the meta argument, then not all columns will populate
# way to grab those dynamically?
# what we really want here is a way to 
metrics = my_bag.pluck('metric').to_dataframe(meta={'data0': str, 'data1': str, 'data2': str, 'data3': str})

In [ ]:
lazy_df1 = my_bag.to_dataframe().join(metrics).drop('metric', axis=1)

In [ ]:
df1 = lazy_df1.compute()

In [ ]:
df1

In [ ]:
lazy_df2 = lazy_df1.explode('values')
values = lazy_df2.pop('values')

In [ ]:
lazy_df2.join(values.to_bag().to_dataframe(meta={'time': int, 'value': str}))

In [ ]:
import numpy as np
import pandas as pd
import dask.dataframe as dd
import dask.bag as db

b = db.from_sequence([{'a': 1,   'b': 1},
                     {'a': np.inf,   'b': np.nan},
                    {'a': 1,   'b': np.nan}],
                    npartitions=2)
df = b.to_dataframe(meta={'a': float, 'b': "Int64"})
df.compute()

In [ ]:
b.pluck?

In [ ]:
db.

In [ ]:
b.__dict__

In [ ]:
b.head()

In [ ]:
b.visualize()

In [ ]:
b.npartitions

In [ ]:
b.partition[0]

In [ ]:
import numpy as np
import pandas as pd
import dask.dataframe as dd
import dask.bag as db

b = db.from_sequence([{'a': 1,   'b': 1},
                     {'a': np.inf,   'b': np.nan},
                    {'a': 1,   'b': np.nan}],
                    npartitions=2)
df = b.to_dataframe(meta=object)
df.compute()

In [ ]:
b.__dict__

In [ ]:
b.name

In [ ]:
df = b.to_dataframe()

In [2]:
import numpy as np
import pandas as pd
import dask.dataframe as dd
import dask.bag as db

b = db.from_sequence([{'a': 1,   'b': 1},
                     {'a': np.inf,   'b': np.nan},
                    {'a': 1,   'b': np.nan}],
                    npartitions=2)
ddf = b.to_dataframe()
my_cols = list(ddf.columns)
meta_dict = dict(zip(my_cols, [object]*len(my_cols)))
ddf = b.to_dataframe(meta_dict)
ddf.compute()

,a,b
0,1.0,1.0
1,inf,NaN
0,1,NaN
